# Bunq Challenge

In [197]:
#base libaries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import re
import warnings
import time

#Jupyter visualization
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
warnings.filterwarnings("ignore")

#advanced libaries
import pickle
import category_encoders as ce
from sklearn.metrics import confusion_matrix, recall_score, precision_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler, Imputer
import lime
from lime import lime_tabular
#rom treeinterpreter import treeinterpreter as ti

#ML libaries
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model, datasets, tree, svm
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor 
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score, classification_report
from sklearn.externals import joblib
#from sklearn import cross_validation
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA

#DL libaries
import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.models import model_from_json
import keras.callbacks as cbks
from keras.constraints import min_max_norm
from keras.constraints import unit_norm


#Progress bars
from tqdm import tqdm_notebook
from tqdm import tqdm 
tqdm.pandas()
#install tqdm: https://towardsdatascience.com/progress-bars-in-python-4b44e8a4c482

In [175]:
#TO-DO: implement one_hot_encoding

def get_dtypes(df, dtype, ignore=[]):
    keys=[]
    for i in range(len(df.keys())):
        if np.array([df[df.keys()[i]]]).dtype == dtype and df.keys()[i] not in ignore:
            keys.append(df.keys()[i])
    return keys

def remove(df, cols):
    df_id = df[cols]
    df=df.drop(cols,axis=1)
    return df_id, df

def encoding(dataframe, catAttributes, encoder):
    RANDOM_SEED = 42 # Life, Universe and Everything.
    if encoder == "onehot":
        enc = ce.OneHotEncoder()
    if encoder == "hashing":
        enc = ce.HashingEncoder()
    toEncode = dataframe[catAttributes].as_matrix()
    enc.fit(toEncode)
    encoded = enc.transform(toEncode)
    encoded = pd.DataFrame(encoded)
    #numerical=pd.DataFrame(dataframe[numAttributes],columns=numAttributes)
    x = pd.merge(dataframe,encoded,left_index=True, right_index=True)
    return(x, enc)

def normalize(dataframe, ignore, scalarfile, save=True):
    table = dataframe[:]
    normalizer = MinMaxScaler()
    #original:
    cols_normalize = list(set(dataframe.keys()) - set(ignore))
    #cols_normalize = list(set(dataframe.keys())) #for debug
    
    #pd.rename() can do that more efficiently.
    #https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html
    cols_renamed = [str(col) + "_n" for col in cols_normalize]    
    table[cols_renamed] = table[cols_normalize] 
    normalizer.fit(table[cols_renamed])
    
    norm_df = pd.DataFrame(normalizer.transform(table[cols_renamed]), 
                                 columns=cols_renamed, 
                                 index=table.index)
    
    join_df = table[table.columns.difference(cols_renamed)].join(norm_df)
    table = join_df.reindex(columns = table.columns)
#    table.head()
    if save:
        joblib.dump(normalizer, "scalars/"+scalarfile+".save")
    return table, scalarfile, cols_normalize

def replace_categorical_nan(df, fields, log = False):
    """Replaces nan with caterical the categorical values present in the dataset
    The categorical  values are randomly choosen and keep the underlying distribution the of non nan values.
    Input:
        df: dataframe with categorical fields 
        fileds: list of the categorical fields 
        log: 
    Output:
    """

    for field in tqdm_notebook(fields):
        #creates a random permuation of the categorical values
        permutation = np.random.permutation(df[field])
        
        #erase the empty values
        empty_is = np.where(permutation == "")
        permutation = np.delete(permutation, empty_is)
        
        #replace all empty values of the dataframe[field]
        end = len(permutation)
        df[field] = df[field].apply(lambda x: permutation[np.random.randint(end)] if pd.isnull(x) else x)
        
        if log:
            print(f"Field: {field} had nans replaced")
        
    return df
    
    
def replace_nan(df, fields, log = False):
    """Replaces nan with numerical values present in the dataset.
    The numbers are randomly choosen so to keep the previous distribution.
    Inputs:
        df: the dataframe to be analized.
        field: fields with numerical values
        log: print a line with the progress, progress bar is always printed.
    Output:
        dataframe with the the nan replaced by numbers
    """
    for i in range(len(fields)):
        random = df[[fields[i],'unique_id']].loc[~df[fields[i]].isnull()]
        df[fields[i]] = df[fields[i]].apply(lambda x: random[fields[i]].loc[np.random.permutation(random.index)[1]] if pd.isnull(x) else x).astype(float)
        if log:
            print('`replace_nan` > processed: ', i+1,'/',len(fields))
    return df

def factorize(df, fields, log=False):
    """Factorizes the non-values of a dataframe into numbers.
    The original data is kept on columns of name+"_uf", in the same dataframe.
    input:
        df: pandas dataframe to be factorized.
        fields: list containing the name of the columns which daa should be factorized.
        log: option to log the progress 
    return:
        df: the modified pandas dataframe.
        fields: a np array containing the name of the columns with the original data.
    """
    if log:
        print("factorizing...")
    
    #fields is expected to be an np.array or a list, 
    #for more efficient execution, it is converted into a np.array dtype object.
    #this datatype allows for the necessary vectorization.
    fields = np.array(fields, dtype = 'object')
    
    #This can probably be done more efficiently by vectorization
    #https://stackoverflow.com/questions/39390160/pandas-factorize-on-an-entire-data-frame/39390208
    
    for field in tqdm(fields): 
        try:
            df[field + '_uf'] = df[field]
            df[field], label = pd.factorize(df[field])
        except KeyError as error:
            #Raise exception when they key is not found in the dataframe.
            print(f"problematic field: {field}")
        if log:    
            print('`factorize` > processed: ', i+1,'/',len(fields), "\r")
    
    #modify the name of the fields to name+"_uf"    
    fields += '_uf'
    return df, fields

def run_non_seq_model(x_train, x_test, y_train, y_test, model, log = False):
    if(model=="LOG"):
        MOD = linear_model.LogisticRegression()
    if(model=="GAUSS"):
        MOD = GaussianNB()
    if(model=="TREE"):
        MOD = tree.DecisionTreeClassifier()
    if(model=="NN"):
        MOD = MLPClassifier(hidden_layer_sizes=(5, 5),solver='sgd')
    if(model=="RF"): #Best Performer so far
        mf = x_train.shape[1]
        MOD = RandomForestClassifier(max_features = mf, #original: 41, 
                                   n_estimators = 10, 
                                   max_depth = 15, 
                                   min_samples_split = 3, 
                                   verbose=2) #use GridSearch to find optimal parameters (section Model optimization)
    
    if(model=="SVM"): #Slow, but the best for recall
        MOD = svm.SVC(kernel = 'linear',probability=True, max_iter=1000)
        #max_iter = 1000 is a good number for a coffee
        
    
    MOD.fit(x_train, y_train)
    y_pred_class = MOD.predict(x_test)
    y_pred_prob =  MOD.predict_proba(x_test)
    if log:
        print(model,"\n confusion matrix:")
        print(confusion_matrix(y_test, y_pred_class))
    accuracy = accuracy_score(y_test,y_pred_class)
    precision = precision_score(y_test,y_pred_class)
    recall = recall_score(y_test,y_pred_class)
    return [accuracy,precision, recall], y_pred_class, y_pred_prob, MOD

def plot_feature_importances(model):
    fig, ax=plt.subplots(dpi=10)
    plt.clf()
    plt.figure(figsize=(18,16))
    n_features = len(x_train_data.keys())
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), x_train_data.keys())
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    plt.ylim(-1, n_features)
    plt.show()

def one_hot_encoding():
    #implement one_hot_encoding using pandas get_dumies
    #https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html
    pass

def check_nan_bilabel_correlation(df, label_column, keys, threshold = .1, log = True):
    """check each columns have their fields empity when label == 0 and filled when label == 1.
    Inputs:
        df: dataframe to be analized.
        label_column: string, the column containing binary labels
        keys: list, the columns of the dataframe to be analized
        threshold: float, defines the correlation, the lower the number the stronger the correlation
        log: boolean, True to print information.
    Return: a list with the problematic columns
    
    """
    
    positives = df_cz[label_column].sum()
    data_size = len(df)
    negatives = data_size - positives
    
    bad_keys = []
    
    for key in keys:
        
        nan_occurences_1 = sum(df.loc[df_cz[label_column] == 1, key].isnull())
        nan_occurences_0 = sum(df.loc[df_cz[label_column] == 0, key].isnull())
        
        if (nan_occurences_1 < threshold*positives) and (nan_occurences_0 > threshold*negatives):
            
            bad_keys.append(key)
            
            if log:
                print(f"column: {key}, number of empity fields: {nan_occurences_1} for label: 1")
                print(f"column: {key}, number of empity fields: {nan_occurences_0} for label: 0")
                print(f"number of unique values: {len(df_cz[key].unique())}")
                print(f"{100*sum(df[key].isnull())/data_size}% of the column is empity")
                print("")
                
    return bad_keys

In [176]:
df_bq = pd.read_csv('data_clean.csv', 
                    header=0, 
                    sep=',', 
                    index_col=0, 
                    parse_dates=True, 
                    encoding='ISO-8859-1', 
                    tupleize_cols=None, 
                    infer_datetime_format=False,
                    error_bad_lines = False)

In [177]:
df_bq.head()

,reporting_start,reporting_end,campaign_id,fb_campaign_id,age,gender,interest1,interest2,interest3,impressions,clicks,spent,total_conversion,approved_conversion
ad_id,,,,,,,,,,,,,,
708746,17/08/2017,17/08/2017,916.0,103916.0,30-34,M,15,17,17,7350,1,1.43,2,1
708749,17/08/2017,17/08/2017,916.0,103917.0,30-34,M,16,19,21,17861,2,1.82,2,0
708771,17/08/2017,17/08/2017,916.0,103920.0,30-34,M,20,25,22,693,0,0.00,1,0
708815,30/08/2017,30/08/2017,916.0,103928.0,30-34,M,28,32,32,4259,1,1.25,1,0
708818,17/08/2017,17/08/2017,916.0,103928.0,30-34,M,28,33,32,4133,1,1.29,1,1


In [178]:
df_bq["conv/spent"] = (df_bq["approved_conversion"])/(df_bq["spent"]+1)
df_bq["conv/(spent*imp)"] = (df_bq["approved_conversion"])/(df_bq["spent"]*df_bq["impressions"]+1)


In [179]:
df_bq.head(100)

,reporting_start,reporting_end,campaign_id,fb_campaign_id,age,gender,interest1,interest2,interest3,impressions,clicks,spent,total_conversion,approved_conversion,conv/spent,conv/(spent*imp)
ad_id,,,,,,,,,,,,,,,,
708746,17/08/2017,17/08/2017,916.0,103916.0,30-34,M,15,17,17,7350,1,1.43,2,1,0.411523,9.513390e-05
708749,17/08/2017,17/08/2017,916.0,103917.0,30-34,M,16,19,21,17861,2,1.82,2,0,0.000000,0.000000e+00
708771,17/08/2017,17/08/2017,916.0,103920.0,30-34,M,20,25,22,693,0,0.00,1,0,0.000000,0.000000e+00
708815,30/08/2017,30/08/2017,916.0,103928.0,30-34,M,28,32,32,4259,1,1.25,1,0,0.000000,0.000000e+00
708818,17/08/2017,17/08/2017,916.0,103928.0,30-34,M,28,33,32,4133,1,1.29,1,1,0.436681,1.875268e-04
708820,17/08/2017,17/08/2017,916.0,103929.0,30-34,M,29,30,30,1915,0,0.00,1,1,1.000000,1.000000e+00
708889,17/08/2017,17/08/2017,916.0,103940.0,30-34,M,15,16,17,15615,3,4.77,1,0,0.000000,0.000000e+00
708895,17/08/2017,17/08/2017,916.0,103941.0,30-34,M,16,20,18,10951,1,1.27,1,1,0.440529,7.189708e-05
708953,17/08/2017,17/08/2017,916.0,103951.0,30-34,M,27,31,31,2355,1,1.50,1,0,0.000000,0.000000e+00


In [180]:
df_bq.columns

Index(['reporting_start', 'reporting_end', 'campaign_id', 'fb_campaign_id',
       'age', 'gender', 'interest1', 'interest2', 'interest3', 'impressions',
       'clicks', 'spent', 'total_conversion', 'approved_conversion',
       'conv/spent', 'conv/(spent*imp)'],
      dtype='object')

In [181]:
# discard: dates, campaign_id, fb_campaign_id
# features: age, gender, interest1,2,3

len(df_bq)

1143

In [182]:
df_bq["reporting_start"].unique()

array(['17/08/2017', '30/08/2017', '29/08/2017', '18/08/2017',
       '19/08/2017', '27/08/2017', '26/08/2017', '25/08/2017',
       '28/08/2017', '20/08/2017', '21/08/2017', '24/08/2017',
       '23/08/2017', '22/08/2017'], dtype=object)

In [183]:
df_bq["conv/spent"].describe()

count    1143.000000
mean        0.111883
std         0.275887
min         0.000000
25%         0.000000
50%         0.004936
75%         0.045935
max         2.000000
Name: conv/spent, dtype: float64

In [184]:
df_bq["interest1"].unique()

array([ 15,  16,  20,  28,  29,  27,  31,   7,  30,  24,  21,  32,  18,
        63,  65,  25,  10,  19,  26,  36,  23,  64,  22,   2,  66, 100,
       101, 102, 103, 105, 107, 110, 111, 112, 113, 108, 109, 114, 104,
       106], dtype=int64)

In [185]:
#pre-process the data
feat_df = df_bq[["age", "gender", "interest1", "interest2", "interest3"]]
feat_df["gender"] = feat_df["gender"].apply(lambda x: 1 if x == "M" else 0)
feat_df["age"] = feat_df["age"].apply(lambda x: int(x[-2:])/2 + int(x[-5:-3])/2)
#todo: one-hot encode or hash the interests

# conv/spent are from 0-2, no need to normalize them.
label_df = df_bq["conv/spent"]


label_df.head()

ad_id
708746    0.411523
708749    0.000000
708771    0.000000
708815    0.000000
708818    0.436681
Name: conv/spent, dtype: float64

In [186]:
#split data into test and train (use random split or split on period)
x_train, x_test, y_train, y_test = train_test_split(feat_df, label_df, test_size=0.3, random_state=42)


#normalize data based on the test set
normalizer = MinMaxScaler()
normalizer.fit(x_train)

x_train = pd.DataFrame(normalizer.transform(x_train),
                       columns = x_train.keys(),
                       index = x_train.index)

#test test is normalized based on test set. 
x_test = pd.DataFrame(normalizer.transform(x_test),
                      columns = x_test.keys(),
                      index = x_test.index)

In [202]:
model = {"LR": LinearRegression(fit_intercept = True)}
    
#     "MLP": MLPRegressor(random_state = 42),
#     "LR": LinearRegression(fit_intercept = True),
#     "GP": GaussianProcessRegressor(random_state = 42),
#     "SVM": LinearSVR(random_state = 24),
#     "RF": RandomForestRegressor(random_state = 42),
#     "DT": DecisionTreeRegressor(random_state = 42)
# }

m_params = {
    "MLP": {"hidden_layer_sizes": [(100,), (100, 100), (20, 50, 100, 50, 20), (100, 50, 20), (20, 50, 100)],
            "activation": ["relu", "identity", "logistic", "tanh"],
            "learning_rate": ["constant", "invscaling", "adaptive"],
            "learning_rate_init": [0.0001, 0.001, 0.01],
            "max_iter": [200, 500, 1000]},
    "LR": {"fit_intercept": [True, False]}, #False should be bad, but just in case...
    "GP": {"kernel": ["RBF"]},
    "SVM": {},
    "RF": {},
    "DT": {}
}

mod = MLPRegressor(hidden_layer_sizes = (100,), random_state = 42)
mod.fit(x_train, y_train)
y_pred = mod.predict(x_test)
#y_pred_prob =  mod.predict_proba(x_test)


In [206]:
search_inter = 20

for m in model:
#    model[m].fit(x_train, y_train)
    random_search = RandomizedSearchCV(model[m], 
                                       param_distributions = m_params[m], 
                                       n_iter = search_inter) #implement that instead of fit.
    
    random_search.fit(x_train, y_train)
    scores = cross_val_score(model[m], x_test, y_test, cv=5)
    print(m, model[m])
    #print(model[m].score(x_test, y_test), scores)
    print("scores", scores)

LR LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)
scores [ 0.02141573 -0.06556579 -0.03986876 -0.01787727  0.09521443]


In [195]:
#final test:
for m in model:
    print("Model type:", m)
    print("Model parameters:", model[m])
    print("Final Score:", model[m].score(x_test, y_test))
    print("______________________________________________")

NotFittedError: This MLPRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.